In [ ]:
# This is required to get the json file
# If you don't want to do this,
# You can directly upload the json file in the colab
# And then give the path accordingly in step4

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip3 install indic_transliteration   # For Transliteration
!pip3 install indicsyllabifier        # For Breaking down the syllables

     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 665kB 31.3MB/s 
     |████████████████████████████████| 911kB 37.2MB/s 
  Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Created wheel for indicsyllabifier: filename=indicsyllabifier-0.3-cp37-none-any.whl size=6098 sha256=63a0e447fffb9c3e1338950b3630b82ec9442eccde423701d2ad0be8e09ccc72
  Stored in directory: /root/.cache/pip/wheels/38/7a/d2/9dbfa38cb27a493067034c25e7b196382520d096cc75caa0b9
  Created wheel for silpa-common: filename=silpa_common-0.3-cp37-none-any.whl size=8482 sha256=17c34a9cf1afd9ea839cb0ae3748498917fff97041a11230a9423a331ac4f1e0
  Stored in directory: /root/.cache/pip/wheels/16/4f/ba/604a82bf904740f1a1d3ad88029c0df5c638bd8825a3cb972d
Successfully built indicsyllabifier silpa-common


In [ ]:
from indic_transliteration import detect, sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
import indicsyllabifier
from itertools import combinations
import json
from time import time, sleep
from tqdm.notebook import tqdm_notebook
from random import choice,choices, randint, sample
import sqlite3

In [ ]:
path = '''/gdrive/MyDrive/HoMI/Encoder - v1/'''
Database = sqlite3.connect(f'''{path}D_Database.db''')
cursor = Database.cursor()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Encodings',)]


In [ ]:
def Encoder(Number):

  options = get_options(Number)

  Normal = []
  Reversed = []

  for option in options:

    if option[2] == 0:
      Normal.append(option[1])
    else:
      Reversed.append(option[1])
  
  return   Normal, Reversed

In [ ]:
def Split_Number(Number,Number_of_sentences):
  scheme_map = SchemeMap(SCHEMES[sanscript.SLP1], SCHEMES[sanscript.DEVANAGARI])  
  l=len(Number)
  Split_pos=[]   ## for storing the places where the number can be split
  Sentence_collection=[]
  i=0
  while i<len(Number)-1:   ## creates a list of the number of places where the input number can be split for eg if i=2 then the number can be split in 2 places
    Split_pos.append(i)
    i=i+1     
  b=[]   ## for storing the split pieces of number 
  n=1 ## the initial number of places at which the number is split
  while len(Sentence_collection)<Number_of_sentences and n<len(Number):   ##  If the number has l digits it can be split in l-1 places
    comb=combinations(Split_pos,n)
    for j in list(comb):
      b.append(Number[0:j[0]+1])
      k=0
      while k<n-1:
        b.append(Number[j[k]+1:j[k+1]+1])
        k=k+1
      b.append(Number[j[n-1]+1:l]) 
      minimum=4    
      c=[]
      Total_number_of_sentences=1
      for part in b:
        Normal,Reversed=Encoder(part)
        Total_number_of_sentences=Total_number_of_sentences*len(Normal)
        c.append(Normal)        
      if Total_number_of_sentences>0:
        i=0
        ls=len(Sentence_collection)
        while (len(Sentence_collection)-ls)<=(Number_of_sentences/(2**(len(Number)-1)-1)) and ((len(Sentence_collection)-ls)<Total_number_of_sentences):
          sentence=[]
          for x in c:
            select=choice(x)
            sentence.append(transliterate(select, scheme_map=scheme_map))
          if sentence not in Sentence_collection:
            Sentence_collection.append(sentence)
      b=[]
    n=n+1
  return Sentence_collection   

In [ ]:
def get_options(Number):

  # Edited  - Hrushikesh
  #cursor.execute(f"select Katapayadi, Text, Is_Reverse from Katapayadi where Katapayadi = {Number}")
  cursor.execute(f"select Katapayadi, Word, Is_Reverse from Encodings where Katapayadi = ?", (Number,))
  return cursor.fetchall()

In [ ]:
def print_Normal(Number,Number_of_words,Number_of_sentences):

  scheme_map = SchemeMap(SCHEMES[sanscript.SLP1], SCHEMES[sanscript.DEVANAGARI])
  Normal,Reversed=Encoder(Number)
  if Number_of_words <= len(Normal):
    Normal = sample(Normal, k=Number_of_words)
    print("\nWords encoded in normal form :")
    i=0
    while i<len(Normal):
      print(f"\t{i+1}.\t{transliterate(Normal[i], scheme_map=scheme_map)}")
      i=i+1
  else: 
    print(f"\nWe have {len(Normal)} words encoded in normal form :")
    i=0
    while i<len(Normal):
      print(f"\t{i+1}.\t{transliterate(Normal[i], scheme_map=scheme_map)}")
      i=i+1
  if Number_of_sentences>0:
    if len(Number)>1:   
      Sentence_collection=Split_Number(Number,Number_of_sentences)
      New_Sentence_collection=[]
      for Sentence in Sentence_collection:
        Sentence = Sentence[::-1]
        New_Sentence_collection.append(Sentence) 
      if len(New_Sentence_collection)>=Number_of_sentences:
        print("Sentences encoded in normal form")
        i=0
        for Sentence in sample(New_Sentence_collection,k=Number_of_sentences): 
          Print_Sentence=""
          for Word in Sentence:
            Print_Sentence=Print_Sentence+" "+ Word  
          print(f"\t{i+1}.\t{Print_Sentence.strip()}")  
          i=i+1
      else:
        print(f"Only {len(New_Sentence_collection)} sentences could be encoded in the normal form")
        i=0
        for Sentence in New_Sentence_collection: 
          Print_Sentence=""
          for Word in Sentence:
            Print_Sentence=Print_Sentence+" "+ Word  
          print(f"\t{i+1}.\t{Print_Sentence.strip()}")  
          i=i+1     
    else:
      print("\n No sentences were generated")

In [ ]:
def print_Reversed(Number,Number_of_words,Number_of_sentences):
  Number=Number[::-1]
  scheme_map = SchemeMap(SCHEMES[sanscript.SLP1], SCHEMES[sanscript.DEVANAGARI])
  Normal,Reversed=Encoder(Number)
  if Number_of_words <= len(Normal):
    Normal = sample(Normal, k=Number_of_words)
    print("\nWords encoded in reverse form :")
    i=0
    while i<len(Normal):
      print(f"\t{i+1}.\t{transliterate(Normal[i], scheme_map=scheme_map)}")
      i=i+1
  else: 
    print(f"\nWe have {len(Normal)} words encoded in reverse form :")
    i=0
    while i<len(Normal):
      print(f"\t{i+1}.\t{transliterate(Normal[i], scheme_map=scheme_map)}")
      i=i+1
  if Number_of_sentences>0:  
    if len(Number)>1:
      Sentence_collection=Split_Number(Number,Number_of_sentences)
      New_Sentence_collection=[]
      for Sentence in Sentence_collection:
        Sentence = Sentence[::-1]
        New_Sentence_collection.append(Sentence) 
      if len(New_Sentence_collection)>=Number_of_sentences:
        print("Sentences encoded in reverse form")
        i=0
        for Sentence in sample(New_Sentence_collection,k=Number_of_sentences): 
          Print_Sentence=""
          for Word in Sentence:
            Print_Sentence=Print_Sentence+" "+ Word  
          print(f"\t{i+1}.\t{Print_Sentence.strip()}")  
          i=i+1
      else:
        print(f"Only {len(New_Sentence_collection)} sentences could be encoded in the reverse form")
        i=0
        for Sentence in New_Sentence_collection: 
          Print_Sentence=""
          for Word in Sentence:
            Print_Sentence=Print_Sentence+" "+ Word  
          print(f"\t{i+1}.\t{Print_Sentence.strip()}")  
          i=i+1
    else:
      print("\n No sentences were generated")       

In [ ]:
def main():

  Number = input("Enter Number :")
  Number_of_words = int(input("\nHow many words to show ? : "))
  Number_of_sentences = int(input("\nHow many sentences to show ? : "))
  start=time()
  print_Normal(Number,Number_of_words,Number_of_sentences)
  print_Reversed(Number,Number_of_words,Number_of_sentences)
  print(f"The Process took {time()-start}")

In [ ]:
main()

Enter Number :56321

How many words to show ? : 30

How many sentences to show ? : 50

We have 3 words encoded in normal form :
	1.	करालतम
	2.	पीठलक्षण
	3.	पीठलक्षण
Sentences encoded in normal form
	1.	पट् रण्डा क्षुण
	2.	कुष् प्रष् लक्षेश
	3.	परिलुल् चाक्ष्म
	4.	स्पृश् खलोक्ति ड्मा
	5.	ख्याञ् क्रीडितु शुच्
	6.	यू ष्रील ताम् मद्
	7.	टीं फौल्लि क्षद् शित्
	8.	ज्येष्ठा बुं स्तीर्णि
	9.	पराडि क्षुण
	10.	कूर्द् गोतमी
	11.	किर् प्रग तृणह्
	12.	पोत्री गतिमत्
	13.	त्र्याव्रित् लक्षय् शप्
	14.	व्यट् रोगषम
	15.	पाठाली स्तन् श्मन्
	16.	कृञ् फलोत्तमा
	17.	पत्त्राङ्ग तूणि
	18.	कत् फलोत्तमा
	19.	कृश् प्र्ॐ गौतमि
	20.	कौष्ठ लोप्तृ ग्मा
	21.	पुरी ग्लैं क्षैं ङीष्
	22.	स्टूरिन् गती मश्
	23.	न्यूह् रण्डा तृणह्
	24.	परिगीति मीढ्
	25.	व्यथ् ह्री डण् तूण
	26.	पेरु लिष् क्षर् माङ्
	27.	पृ त्रै प्लुषि ङीन्
	28.	कण्ठील चिमि
	29.	याट् श्रीगुप्त ज्मन्
	30.	कञ् प्रोह् गीत शील्
	31.	कूर क्लौं क्षीण
	32.	पाद् प्राग्लक्षण
	33.	ज्यैष्ठी गौतमि
	34.	पारी ग्लातृ शिष्
	35.	ज्यैष्ठी क्लूं क्षणिन्
	36.	कण्ठ बल् ता मृत्
	

In [ ]:
main()

NameError: ignored